In [3]:
# import numpy, sympy, configure printing and plotting
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
#import seaborn as sns

In [10]:
# notebook config
#sns.set()                                   # nice plotting defaults
sym.init_printing(use_latex='mathjax')      # render latex for output
%matplotlib inline
#%config InlineBackend.figure_format = 'svg' # vectorized plots instead of png

# Solving the 1D Heat Equation with `SymPy`

We are interested in solving the system

$$-\frac{d^2 u}{dx^2} + \mu u = 0$$

on the domain $\Omega = (0,L)$ where $L=1$ subject to BCs

$$u(0) = 1,\, \frac{du}{dx}\bigg|_L= 0$$

## finding the exact solution

This type of ODE has a commonly known general solution, $c_{1} e^{\sqrt{\mu} x} + c_{2} e^{- \sqrt{\mu} x}$. Just in case I remembered it incorrectly or made a typo, let's have `SymPy` check that it does indeed satisfy the ODE above; the `assert` statement will raise an error if the proceeding expression does not evaluate to `True`.

    √μ⋅x       -√μ⋅x
c₁⋅ℯ     + c₂⋅ℯ     

We can apply the boundary conditions algebraically to solve for $c_1,\, c_2$

⎛              2⋅√μ  ⎞
⎜    1        ℯ      ⎟
⎜─────────, ─────────⎟
⎜ 2⋅√μ       2⋅√μ    ⎟
⎝ℯ     + 1  ℯ     + 1⎠

Now that we have solved the linear system for $c_1,\, c_2$, we can get the exact solution by substituting for the unknown coefficients.

⎛ 2⋅√μ    2⋅√μ⋅x⎞  -√μ⋅x
⎝ℯ     + ℯ      ⎠⋅ℯ     
────────────────────────
        2⋅√μ            
       ℯ     + 1        

Just to be sure, we let `SymPy` check that our final result satisfies the original ODE as well as each of the boundary conditions. We use the `assert` statement, which will raise an `AssertError` if the following statement does not evaluate to `True`.

If we have a corresponding `*.tex` document to which we want to add the exact solution, we don't want to transcribe the solution by hand-- this is time-consuming and error-prone; we can instead ask `SymPy` for the latex output directly.

One of the downsides of symbolic manipulation is that substitution is __slow__; evaluating our exact solution numerically over our domain of interest would be expensive if done symbolically. Fortunately, `SymPy` implements a `lambdify` method which allows us to create a vectorized function that can numerically evaluate a symbolic expression quickly at many points by leveraging `NumPy`. Since we are done finding our exact solution, let's build a function which takes as input a vector `x` and $\mu$ `mu` and returns the exact solution.

Now we've done the symbolic heavy lifting; we now query our exact solution for differing values of $\mu$ like any other `NumPy` function  and move on. As a last step, let's use `u_exact` to plot the exact solution numerically.

## (a) symbolic galerkin approximation

Suppose we were unable to find an exact solution; we might want to compute the galerkin approximation symbolically as part of the investigation, since this is a 1D problem. 

We can express the weak form of the problem as searching for $u \in \tilde{X}$ such that $a(u,v) = f(v)\,\, \forall v\in \tilde{X}$, where 

$$    a\left( w,\,v \right) = \int_{0}^{L} \left( \frac{dw}{dx} \frac{dv}{dx} + \mu_0 w v \right)\,dx \label{1:wfa}
$$

and $f(v)=0$ since we have no interior forcing ( $f_\Omega = 0$ ).

If we are searching for the solution $u$ tested for __all__ $v\in\tilde{X}$ such that $a(u_G, v) = f(v)$, then we test $u$ against each basis function $\psi_i$, forming a linear system.

We take $u_G = \alpha_i \psi_i$, and since we have an inhomogenous Dirichlet boundary condition, we can write the problem as:

$$a\left(u_G^0, v\right) = f(v) - a\left(w^E_n, v\right)$$

For choice of lifting function $w^E_n$.

We can write two functions which leverage `SymPy` and the weak forms above to solve the linear system for the galerkin coefficients.

#### I. galerkin approximation $u_G^I$

The first galerkin approximation $u_G^{I}$ we consider $\tilde{X} = \text{span}\left\{ x\right\},\, \psi_1 = x$ with the lifting function $w^E_n = 1$. We can create a dictionary of our exact galerkin solutions,  with the value as the symbolic galerkin expression, indexed by the value of $\mu$.

Let's see how the galerkin solutions compare to the exact solution. 

Note that we turn each `SymPy` expression into a lambdified expression, just like we did for the exact solution.

#### II. galerkin approximation $u_G^{II}$

The second galerkin approximation $u_G^{II}$ we consider $\tilde{X} = \text{span}\left\{ x\right\},\, \psi_1 = x$ with the lifting function $w^E_n = \exp\left(-\sqrt{\mu}x\right)$. We can create a dictionary of our exact galerkin solutions,  with the value as the symbolic galerkin expression, indexed by the value of $\mu$. We can plot these by calling the same function above.

In [1]:
# plot the solution

#### III. galerkin approximation $u_G^{III}$

For fun, how would a second order approximation do with a constant lifting function? It's not really any additional work, since our code is general in its input w/r/t basis.

The third galerkin approximation $u_G^{III}$ we consider $\tilde{X} = \text{span}\left\{ x,\, x^2\right\},\, \psi_1 = x,\, \psi_2=x^2$ with the lifting function $w^E_n = 1$. We can create a dictionary of our exact galerkin solutions,  with the value as the symbolic galerkin expression, indexed by the value of $\mu$. We can plot these by calling the same function above.

#### error considerations

It's pretty clear from the output above that $u_G^{II}$ is going to have a much smaller error than $u_G^{I}$ as measured in the energy norm, because we made plots along with the galerkin approximations. 

But suppose we had to choose which galerkin approximation was best without reference to the exact solution. How could we do it?

We seek the approximation which provides the smallest error in the energy norm. Note that $a(w,w) = \int_{0}^{L} \left( \frac{dw}{dx}
    \right)^{2} + \mu_0 w^2 dx$ and $f(v) = 0$ hence 
    
$$\Pi(w) = \frac{1}{2}\int_{\Omega}\left[ \left(
      \frac{dw}{dx} \right)^2 + \mu_0 w^2 \right]\, dx$$

The galerkin approximations $u_G$ have the nice property that $\Pi(u^*) \leq \Pi(u_G)$ for any $u_G$. In other words, that the exact solution $u^*$ is the argument which minimizes the energy functional $\Pi$, and any galerkin solution will have a greater "energy".

We can simply take our galerkin solutions $u_G^{I},\, u_G^{II}$ and evaluate the "energies" $\Pi(u_G)$ symbolically to see which approximation has the smaller "energy."

Consider $\mu_{0} = \frac{1}{10},\, 1,\, 10$ and $n=1$. 

We can evaluate $\Pi(u_G)$ for each galerkin approximation, $\mu$ pair symbolically. Before printing, we cast the exact symbolic expressions to floating point numbers (we print to 3 significant digits) for ease of visual comparison. 

As we expected, galerkin approximation $u_G^{II}$ has a smaller error in the energy norm than does $u_G^{I}$.